<a href="https://colab.research.google.com/github/khal-drog0/Breast_Cancer_Histopathology_Classification/blob/main/BeforeVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip3 install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json /root/.kaggle

! chmod 600 /root/.kaggle/kaggle.json

! kaggle datasets download -d paultimothymooney/breast-histopathology-images

! unzip breast-histopathology-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating: IDC_regular_

In [1]:
import numpy as np
import pandas as pd
import os
import itertools
import os, stat, time
from os.path import dirname as up
import shutil
import glob
from PIL import Image
import gc

In [2]:
from glob import glob
data = glob('IDC_regular_ps50_idx5/**/*.png', recursive = True)

In [3]:
# import cv2
# import matplotlib.pyplot as plt

# for i in data[:5]:
#     img = cv2.imread(i)
#     img_1 = cv2.resize(img, (200, 200))
#     plt.imshow(img_1, cmap = 'binary')
#     plt.show()

In [4]:
import random

def randomSamples(imageData):
    lst = list()
    while(len(lst) != 15000):
        a = random.choice(imageData)

        if a not in lst:
            lst.append(a)

    return lst

In [5]:
import cv2
import matplotlib.pyplot as plt

images = list()
labels = list()
newData = list()

newData = randomSamples(data)

for i in newData:
    if i.endswith('.png'):
        label = i[-5]
        img = cv2.imread(i)
        img_1 = cv2.resize(img, (100, 100))
        images.append(img_1)
        labels.append(label)

In [6]:
del newData
gc.collect()

140

In [7]:
len(images)

15000

In [8]:
x = np.stack(images)

In [9]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(labels)

In [10]:
x = x / 255

In [12]:
del images
del labels
gc.collect()

140

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0, test_size = 0.2)

In [13]:
print('len(x_train):', len(x_train))
print('len(x_test):', len(x_test))
print('len(y_train):', len(y_train))
print('len(y_test):', len(y_test))

len(x_train): 12000
len(x_test): 3000
len(y_train): 12000
len(y_test): 3000


# Building the Neural Network

In [14]:
from tensorflow.keras.layers import Dense, Dropout # , BatchNormalization, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential
# from tensorflow.keras import regularizers
# from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet import EfficientNetB2
# from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import models, layers
# from tensorflow.keras.regularizers import L2

# model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2()

# def modelInceptionResNetV2():
def modelEfficientNetB2():
    model = models.Sequential()
    # model.add(EfficientNetB0(include_top = False, weights = 'imagenet', input_shape = (100, 100, 3)))
    # model.add(InceptionResNetV2(include_top = False, weights = 'imagenet', input_shape = (100, 100, 3)))
    model.add(EfficientNetB2(include_top = False, weights = 'imagenet', input_shape = (100, 100, 3)))
    # model.add(layers.Dense(128, activation = 'relu', kernel_regularizer = L2(0.00001)))
    model.add(layers.Dropout(0.05))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(2, activation = 'sigmoid'))

    return model

model = modelEfficientNetB2()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb2 (Functional)  (None, 4, 4, 1408)       7768569   
                                                                 
 dropout (Dropout)           (None, 4, 4, 1408)        0         
                                                                 
 global_average_pooling2d (G  (None, 1408)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 2818      
                                                                 
Total params: 7,771,387
Trainable params: 7,703,812
Non-trainable params: 67,575
_________________________________________________________________


In [15]:
from tensorflow.keras.optimizers import Adam

opt = Adam(lr = 0.00001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad =  False)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Training the model

In [ ]:
history = model.fit(x, y, epochs = 10, validation_split = 0.4, batch_size = 64)

Epoch 1/10
141/141 [==============================] - ETA: 0s - loss: 0.5994 - acc: 0.7227

# Evaluating the model

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)

In [ ]:
# Save the model
model.save('before.h5')
# new_model = tensorflow.keras.models.load_model('before.h5')
# new_model.summary()

In [ ]:
plt.figure(figsize = (12, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train_data', 'test_data'])
plt.title('Loss Analysis')
plt.show()

In [ ]:
plt.figure(figsize = (12, 5))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train_data', 'test_data'])
plt.title('Accuracy Analysis')
plt.show()

# Testing with random inputs

In [ ]:
IMG = Image.open('9383/0/9383_idx5_x1001_y1501_class0.png')
print(type(IMG))

IMG = IMG.resize((100, 100))
IMG = np.array(IMG)
print('po array = {}'.format(IMG.shape))

IMG = np.true_divide(IMG, 255)
IMG = IMG.reshape(-1, 100, 100, 3)
print(type(IMG), IMG.shape)

y_prob = model.predict(x) 
y_classes = y_prob.argmax(axis=-1)

# predictions = model.predict_classes(IMG)
print(model)
print(y_classes)

# predictions_c = model.predict_classes(IMG)
# print(predictions, predictions_c)

# model.predict_classes(IMG)
model.predict(IMG)
# print(predictions_c)

In [ ]:
classes = {'TRAIN': ['Non Malignant (No Cancer)','Malignant'],
           'TEST': ['Non Malignant (No Cancer)', 'Malignant']}

predicted_class = classes['TRAIN'][predicitions_c[0]]
print('We think that is {}'.format(predicted_class.lower()))

# Saving the model

In [ ]:
model.save_weights('_beforeVAE_.h5')

model_json = model.to_json()
with open('_beforeVAE_.json', 'w') as json_file:
    json_file.write(model_json)

print('Model saved to the disk.')